In [ ]:

for text in [user history,,,,,,,,]:
    similarity = text | [big 5 161 문장 vector]
    
    max(similarity, ) # 최고 성향 1개만 
    if similarity > threshold: # 성향정보 저장
        ...
        ranking()  # O: 3, C: 4, E: 7, 
    

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/stsb-xlm-r-multilingual")
model = AutoModel.from_pretrained("sentence-transformers/stsb-xlm-r-multilingual")

In [2]:
def sentence_XLM_similarity(text1, text2):
    sentences = [text1, text2]
    #Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
    
    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        
    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
#     a_norm = torch.nn.functional.normalize(sentence_embeddings[0], p=2, dim=0)
#     b_norm = torch.nn.functional.normalize(sentence_embeddings[1], p=2, dim=0)
    
    similarity = torch.nn.CosineSimilarity(dim=0)(sentence_embeddings[0], sentence_embeddings[1])
#     similarity = torch.mm(a_norm, b_norm.transpose())
    return similarity

In [3]:
sentence_XLM_similarity('나는 사람 만나는 것을 좋아해서 동호회도 자주 다녀', '나는 사람 만나는 것을 좋아하지 않아')

tensor(0.2891)

In [4]:
sentence_XLM_similarity('나는 사람 만나는 것을 좋아해서 동호회도 자주 다녀', '나는 사람들을 만나면 대화를 먼저 시작하는 편이다.')

tensor(0.5418)

In [100]:
sentence_XLM_similarity('난 치킨을 좋아해', 'I like chicken')

tensor(0.9878)

In [108]:
sentence_XLM_similarity('BBQ에서 파는 치킨을 좋아해', 'BBQ sell fried chicken. I like this')

tensor(0.9447)

In [101]:
sentence_XLM_similarity('난 치킨을 좋아해', 'I love chicken')

tensor(0.9579)

In [103]:
sentence_XLM_similarity('치킨을 좋아하는 나', 'I like chicken')

tensor(0.4244)

In [104]:
sentence_XLM_similarity('그녀는 예쁘다', 'she is beautiful')

tensor(0.9766)

In [106]:
sentence_XLM_similarity('그녀는 아름다운 사람이야', 'she is beautiful')

tensor(0.9726)

In [22]:
sentence_XLM_similarity('나는 사람 만나는 것을 좋아해서 동호회도 자주 다녀', '관심의 대상이 되는 것을 꺼리지 않는다.')

tensor(0.6024)

In [96]:
sentence_XLM_similarity('역시 치킨은 맛있어', '역시 치킨이야. 맛이 없을수가 없지')

TypeError: forward() got an unexpected keyword argument 'input_ids'

In [10]:
sentence_XLM_similarity('역시 치킨은 맛없어', '치킨 맛없엉')

tensor(0.9639)

In [91]:
text1 = '치킨은 맛있다'
text2 = '치킨은 맛없다'
sentences = [text1, text2]
#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

TypeError: forward() got an unexpected keyword argument 'input_ids'

In [92]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.4579
A man is playing guitar 		 A woman watches TV 		 Score: 0.1759
The new movie is awesome 		 The new movie is so great 		 Score: 0.9283


In [95]:
embeddings1[0].min()

tensor(-0.6731)